In [11]:
'''
Transforms can be applied to PIL images, tensors, ndarrays, or custom data
during creation of the DataSet
complete list of built-in transforms: 
https://pytorch.org/docs/stable/torchvision/transforms.html
On Images
---------
CenterCrop, Grayscale, Pad, RandomAffine
RandomCrop, RandomHorizontalFlip, RandomRotation
Resize, Scale
On Tensors
----------
LinearTransformation, Normalize, RandomErasing
Conversion
----------
ToPILImage: from tensor or ndrarray
ToTensor : from numpy.ndarray or PILImage
Generic
-------
Use Lambda 
Custom
------
Write own class
Compose multiple Transforms
---------------------------
composed = transforms.Compose([Rescale(256),
                               RandomCrop(224)])
'''
import torch
import torchvision
from torch.utils.data import Dataset

In [17]:
class WineDataset(Dataset):

    def __init__(self, transform=None):
        # Initialize data, download, etc.
        # read with numpy or pandas
        xy = np.loadtxt('wine.csv', delimiter=',', dtype=np.float32, skiprows=1)
        self.n_samples = xy.shape[0]

        # here the first column is the class label, the rest are the features
        self.x_data = xy[:, 1:] # size [n_samples, n_features]
        self.y_data = xy[:, [0]]  # size [n_samples, 1]
        
        self.transform = transform
        
    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        sample = self.x_data[index], self.y_data[index]
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample
        
    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

In [18]:
class ToTensor:
    def __call__(self, sample):
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)

In [22]:
class MulTransform:
    def __init__(self, factor):
        self.factor = factor
        
    def __call__(self, sample):
        inputs, target = sample
        inputs *= self.factor
        
        return inputs, target

In [35]:
dataset = WineDataset(transform=None)

In [36]:
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [37]:
print(features)

[1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00 2.800e-01
 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03 1.000e+00]


In [41]:
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(4)])
dataset = WineDataset(transform=composed)

In [42]:
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [43]:
print(features)

tensor([6.8400e+00, 9.7200e+00, 6.2400e+01, 5.0800e+02, 1.1200e+01, 1.2240e+01,
        1.1200e+00, 9.1600e+00, 2.2560e+01, 4.1600e+00, 1.5680e+01, 4.2600e+03,
        4.0000e+00])
